# [Quantization aware training in Keras example](https://www.tensorflow.org/model_optimization/guide/quantization/training_example)

## Summary
In this tutorial, you will:

1. Train a keras model for MNIST from scratch.
2. Fine tune the model by applying the quantization aware training API, see the accuracy, and export a quantization aware model.
3. Use the model to create an actually quantized model for the TFLite backend.
4. See the persistence of accuracy in TFLite and a 4x smaller model. To see the latency benefits on mobile, try out the TFLite examples in the [TFLite app repository](https://www.tensorflow.org/lite/models)

In [ ]:
!pip install -q tensorflow
!pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import tempfile
import os

import tensorflow as tf
from tensorflow import keras

## Train a model for MNIST without quantization aware training

The **MNIST** database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. The database is also widely used for training and testing in the field of machine learning.